In [1]:
import loopy as lp
import pyopencl as cl
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)
import numpy as np

In [8]:
knl1 = lp.make_kernel("{[i]: 0<=i<n}", "out[i] = 2*input[i]")
knl2 = lp.make_kernel("{[i]: 0<=i<n}", "out2[i] = 5+out[i]")


/home/andreas/src/env-3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: LoopyWarning: 'lang_version' was not passed to make_kernel(). To avoid this warning, pass lang_version=(2018, 2) in this invocation. (Or say 'from loopy.version import LOOPY_USE_LANGUAGE_VERSION_2018_2' in the global scope of the calling frame.)
  """Entry point for launching an IPython kernel.
/home/andreas/src/env-3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: LoopyWarning: 'lang_version' was not passed to make_kernel(). To avoid this warning, pass lang_version=(2018, 2) in this invocation. (Or say 'from loopy.version import LOOPY_USE_LANGUAGE_VERSION_2018_2' in the global scope of the calling frame.)
  


In [13]:
knl1_s = lp.assume(knl1, "n mod 4 = 0 and n>0")
knl1_s = lp.split_iname(knl1_s, "i", 4)

knl1_s = lp.split_array_axis(knl1_s,"out, input", 0, 4)
knl1_s  = lp.tag_inames(knl1_s, "i_inner:vec")
knl1_s  = lp.tag_array_axes(knl1_s, "out,input", "c,vec")
knl1_s = lp.set_options(knl1_s, write_cl=True)
print(knl1_s)

---------------------------------------------------------------------------
KERNEL: loopy_kernel
---------------------------------------------------------------------------
ARGUMENTS:
input: type: <auto/runtime>, shape: ((-1)*(((-1)*n) // 4), 4), dim_tags: (N0:stride:4, vec) aspace: global
n: ValueArg, type: <auto/runtime>
out: type: <auto/runtime>, shape: ((-1)*(((-1)*n) // 4), 4), dim_tags: (N0:stride:4, vec) aspace: global
---------------------------------------------------------------------------
DOMAINS:
[n] -> { [i_outer, i_inner] : i_inner >= 0 and -4i_outer <= i_inner <= 3 and i_inner < n - 4i_outer }
---------------------------------------------------------------------------
INAME IMPLEMENTATION TAGS:
i_inner: vec
i_outer: None
---------------------------------------------------------------------------
INSTRUCTIONS:
for i_inner, i_outer
    out[i_outer, i_inner] = 2*input[i_outer, i_inner]  {id=insn}
end i_inner, i_outer
--------------------------------------------------------

In [14]:
knl1_s(queue, input=np.random.randn(1234*4).astype(np.float32), n=1234*4)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global float4 const *__restrict__ input, int const n, __global float4 *__restrict__ out)
{
  for (int i_outer = 0; i_outer <= ((-4 + n) / 4); ++i_outer)
    out[i_outer] = 2.0f * input[i_outer];
}



TypeError: shape mismatch on argument 'input' (got: (4936,), expected: (1234, 4))